In [2]:
import json

human_test_set = [105,888,673,177,818,183,700,592,682,622,993,922,936,218,360,826,512,131,897,743]

persp_data_path = "../data/dataset/perspective_pool_v0.2.json"
claim_data_path = "../data/dataset/perspectrum_with_answers_v0.2.json"

persps = json.load(open(persp_data_path))
claims = json.load(open(claim_data_path))

persp_dict = {}
claim_dict = {}
for p in persps:
    persp_dict[p["pId"]] = p["text"]

for c in claims:
    claim_dict[c["cId"]] = c

In [12]:
# Make gold annotation
data = []

for cid in human_test_set:
    cur_c = claim_dict[cid]
    c_text = cur_c['text']
    
    p_clusters = [(p['pids'], p['stance_label_3']) for p in cur_c['perspectives']]
    
    p_title_clusters = []
    for c, stance in p_clusters:
        c = [(pid, persp_dict[pid]) for pid in c]
        p_title_clusters.append((c, stance)) 
        
    data.append({
        "claim_id": cid,
        "claim_text": c_text,
        "gold_perspectives" : p_title_clusters
    })
    
out_path = "../data/dataset/human_eval/equivalence_human_eval_gold.json"
json.dump(data, open(out_path, 'w'))
    

In [16]:
# Make the table for stance annotation
import csv

out_path = "../data/dataset/human_eval/stance_human_eval.csv"

f = open(out_path, 'w')
fieldnames = ['claim_id', 'claim_title', 'perspective_id', 'perpsective_title', 'gold_stance']

writer = csv.DictWriter(f, fieldnames=fieldnames)
writer.writeheader()

for c in data:
    
    for clusters in c["gold_perspectives"]:
        stance = clusters[1]
        for p, title in clusters[0]:
            writer.writerow({
                "claim_id": c["claim_id"],
                "claim_title": c["claim_text"],
                "perspective_id": p,
                "perpsective_title": title,
                "gold_stance": stance
            })


In [30]:
# Make spreadsheet for equivalence
import csv
import itertools

out_path = "../data/dataset/human_eval/equivalence_human_eval.csv"

f = open(out_path, 'w')
fieldnames = ['claim_id', 'claim_title', 'perspective_id_1', 'perpsective_title_1', 'perspective_id_2', 'perpsective_title_2']

writer = csv.DictWriter(f, fieldnames=fieldnames)
writer.writeheader()

for c in data:
    all_p = []

    for clusters in c["gold_perspectives"]:
        for p, title in clusters[0]:
            all_p.append((p, title))
    
    l_p = len(all_p)
    
    cartesian = list(itertools.combinations(all_p,2))
        
    for p1, p2 in cartesian:
                
        writer.writerow({
            "claim_id": c["claim_id"],
            "claim_title": c["claim_text"],
            "perspective_id_1": p1[0],
            "perpsective_title_1": p1[1],
            "perspective_id_2": p2[0],
            "perpsective_title_2": p2[1],
        })

In [57]:
# Let's see how our two annotators, Rick and Daniel did!
import numpy as np
import pandas as pd
import json

def is_equivalent_in_gold(pid1, pid2, gold_clusters):
    result = False
    for c in gold_clusters:
        if (pid1 in c) and (pid2 in c):
            result = True
            break
    return result

def compute_p_r_f1(df):
        
    tp = fp = tn = fn = 0
    for idx, row in df.iterrows():
        cid = row.claim_id
        pid1 = row.perspective_id_1
        pid2 = row.perspective_id_2
        
        
        pred_eq = not df.isnull().ix[idx,'Equivalent']
        gold_eq = is_equivalent_in_gold(pid1, pid2, gold_cluster_dict[cid])
        
        if pred_eq and gold_eq:
            tp += 1
        elif pred_eq and not gold_eq:
            fp += 1
        elif not pred_eq and gold_eq:
            fn += 1
        else:
            tn += 1
            
    
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)

    f1 = 2 * prec * rec / (prec + rec)
    return (prec, rec, f1)

rick_result_path = '../data/dataset/human_eval/equivalence_rick.csv'
daniel_result_path = '../data/dataset/human_eval/equivalence_daniel.csv'
gold_path = '../data/dataset/human_eval/equivalence_human_eval_gold.json'

rdf = pd.read_csv(rick_result_path)
ddf = pd.read_csv(daniel_result_path)

gold_data = json.load(open(gold_path))
gold_cluster_dict = {}
for data in gold_data:
    clusters = []
    for c in data["gold_perspectives"]:
        clusters.append([p[0] for p in c[0]])
        
    gold_cluster_dict[data['claim_id']] = clusters
    
# # We only got to row 718...
rdf = rdf[rdf.index < 718]
ddf = ddf[ddf.index < 718]


print(compute_p_r_f1(rdf))
print(compute_p_r_f1(ddf))


/home/squirrel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


(0.8695652173913043, 0.8333333333333334, 0.851063829787234)
(0.8809523809523809, 0.7708333333333334, 0.8222222222222222)
